In [6]:
# Importamos librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler,  OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump

In [11]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Cargamos los datos
Data = pd.read_csv("../Datos/data_adults.csv")

Data_cop = Data.drop("fnlwgt", axis=1)
Data_cop = Data_cop.drop("education-num", axis=1)

X = Data_cop.drop("income", axis=1)
y = Data_cop['income'].isin(['>50K.','>50K'])

# Definimos las variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Creamos pipelines de preprocesamiento
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Usamos ColumnTransformer para combinar ambas transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [12]:
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [13]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos un nuevo pipeline que incluye el preprocesamiento y el modelo
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', LogisticRegression(random_state=42))])

# Entrenamos el pipeline completo en los datos de entrenamiento
model_pipeline.fit(X_train, y_train)

# Hacemos predicciones en el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

# Evaluamos el rendimiento del modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.87      0.93      0.90      7414
        True       0.74      0.58      0.65      2355

    accuracy                           0.85      9769
   macro avg       0.81      0.76      0.78      9769
weighted avg       0.84      0.85      0.84      9769



In [14]:
# Guardamos el pipeline completo (preprocesamiento + modelo entrenado)
dump(model_pipeline, 'model_pipeline_logr.joblib')

['model_pipeline_logr.joblib']

## Optimización de hiperparámetros 

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Cargamos los datos
Data = pd.read_csv("../Datos/data_adults.csv")

Data_cop = Data.drop("fnlwgt", axis=1)
Data_cop = Data_cop.drop("education-num", axis=1)

X = Data_cop.drop("income", axis=1)
y = Data_cop['income'].isin(['>50K.','>50K'])

# Definimos las variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Creamos pipelines de preprocesamiento
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Usamos ColumnTransformer para combinar ambas transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],  
    'solver': ['liblinear', 'saga']  
}

model_logr = LogisticRegression(random_state = 2024)

##Instanciar la busqueda del mejor modelo
model_busqueda = GridSearchCV(estimator=model_logr, 
                              param_grid = param_grid, 
                              cv= 3, 
                              verbose = 4,
                              scoring = 'f1',
                              n_jobs=-1)

model_pipeline_train = Pipeline(steps=[('preprocessor', preprocessor),
                       ('busqueda', model_busqueda)])

model_pipeline_train.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', 'capital-gain', 'capital-loss', 'hours-per-week'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('busqueda',
                 GridSearchCV(cv=3,
                              estimator=LogisticRegression(random_state=2024),
                              n_jobs=-1,
                              param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                                          'penalty': ['l1', 'l2'],
                                          'solver': ['liblinear', 'saga']},
                              scoring='f1', verbose=4))])

In [9]:
busqueda_resultado = model_pipeline_train['busqueda']
busqueda_resultado.best_estimator_.get_params()

{'C': 1,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l1',
 'random_state': 2024,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [10]:
y_pred = model_pipeline_train.predict(X_test)

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.87      0.93      0.90      7414
        True       0.74      0.58      0.65      2355

    accuracy                           0.85      9769
   macro avg       0.81      0.76      0.78      9769
weighted avg       0.84      0.85      0.84      9769



In [11]:
y_pred_train = model_pipeline_train.predict(X_train)

print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

       False       0.88      0.93      0.91     29741
        True       0.73      0.60      0.66      9332

    accuracy                           0.85     39073
   macro avg       0.81      0.76      0.78     39073
weighted avg       0.85      0.85      0.85     39073

